In [1]:
import sys
import os

from transformers import AutoTokenizer, AutoModelForCausalLM
import datasets
from functools import partial

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.utils.dataset_tokenization import process_data

In [2]:
model_path = "../../self-corrective-llama_untrained"
# model_name = "MathBite/self_corrective_llama_3.1_8B_untrained"
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

print(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

SelfCorrectiveLlama(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_em

In [3]:
# SPECIAL_INSTRUCTION = "\nAs you write your answer, you can correct yourself using these tools: Use <DEL_W> to take back the word before this token, <DEL_S> to remove the entire sentence before this token, and <DEL_A> to scrap everything you've written and start again."
# INSERTION_MARKER = "<|start_header_id|>user<|end_header_id|>"
# DELETION_MARKERS = ["<DEL_W>", "<DEL_S>", "<DEL_A>"]
# DELETION_TOKEN_IDS = tokenizer.convert_tokens_to_ids(DELETION_MARKERS)

In [4]:
# data_path = "../../dataset/train.json"
# dataset = datasets.load_dataset("json", data_files=data_path)

In [5]:
# dataset

In [6]:
# sample = dataset["train"][7295]
# print(sample)
# print(sample["correct_response"])
# print(sample["errors"])
# print(sample["hallucinated_text"])

In [7]:
# for i in range(7295, 7296):
#     sample = dataset["train"][i]
    
#     res = process_data(sample, tokenizer, SPECIAL_INSTRUCTION, INSERTION_MARKER, DELETION_TOKEN_IDS[0], DELETION_TOKEN_IDS[1], DELETION_TOKEN_IDS[2])

#     hall_text_idx = [i for i, label in enumerate(res["hallucination_labels"]) if label == 1]
#     hall_text = [res["input_ids"][i] for i in hall_text_idx]
    
#     print(sample["correct_response"])
#     print("================================================")
#     for error in sample["hallucinated_text"]:
#         print(error)
#     print("================================================")
#     print(tokenizer.decode(hall_text))
#     print("--------------------------------\n\n")

In [8]:
# SPECIAL_INSTRUCTION = "\nAs you write your answer, you can correct yourself using these tools: Use <DEL_W> to take back the word before this token, <DEL_S> to remove the entire sentence before this token, and <DEL_A> to scrap everything you've written and start again."
# INSERTION_MARKER = "<|start_header_id|>user<|end_header_id|>"
# DELETION_MARKERS = ["<DEL_W>", "<DEL_S>", "<DEL_A>"]
# DELETION_TOKEN_IDS = tokenizer.convert_tokens_to_ids(DELETION_MARKERS)

# mapper = partial(
#     process_data,
#     tokenizer=tokenizer,
#     special_instruction=SPECIAL_INSTRUCTION,
#     insertion_marker=INSERTION_MARKER,
#     del_w_token_id=DELETION_TOKEN_IDS[0],
#     del_s_token_id=DELETION_TOKEN_IDS[1],
#     del_a_token_id=DELETION_TOKEN_IDS[2]
# )

In [9]:
# dataset.cleanup_cache_files()

In [10]:
# tokenized_dataset = dataset.map(mapper, batched=False, load_from_cache_file=False)
# tokenized_dataset = tokenized_dataset["train"]
# columns_to_remove = [
#     "input", "correct_response", "incorrect_response", 
#     "additional_info", "errors", "hallucinated_text"
# ]

# tokenized_dataset = tokenized_dataset.remove_columns(columns_to_remove)


In [11]:
# tokenized_dataset

In [12]:
# split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
# print(split_dataset)
# train_dataset = split_dataset['train']
# eval_dataset = split_dataset['test']

In [13]:
# output_dir = "../../dataset/training"
# split_dataset.save_to_disk(output_dir)

In [14]:
dataset = datasets.load_from_disk("../../dataset/training")
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'hallucination_labels'],
    num_rows: 36684
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'hallucination_labels'],
    num_rows: 4077
})


In [15]:
# import torch

In [16]:
# sample = train_dataset[5]
# sample["input_ids"] = torch.tensor(sample["input_ids"]).reshape(1, -1)
# sample["hallucination_labels"] = torch.tensor(sample["hallucination_labels"]).reshape(1, -1)
# sample["labels"] = torch.tensor(sample["labels"]).reshape(1, -1)

In [17]:
# sample

In [18]:
# print(sample["labels"][sample["labels"] != -100])
# print(sample["hallucination_labels"][sample["hallucination_labels"] != -100])

In [19]:
# model.forward(
#     input_ids = sample["input_ids"],
#     hallucination_labels = sample["hallucination_labels"],
#     labels = sample["labels"]
# )

In [20]:
# del_tokens = ["<DEL_W>", "<DEL_S>", "<DEL_A>"]


# for i in range(100, 200):
#     sample = train_dataset[i]
#     print(tokenizer.decode(sample["input_ids"]), "\n")

#     for j in range(1, 4):
#         hall_text_idx = [i for i, label in enumerate(sample["hallucination_labels"]) if label == j]
#         hall_text = [sample["input_ids"][i] for i in hall_text_idx]

#         print("--------------------------------\n")
#         print(f"Deletion token: {del_tokens[j-1]}")
#         print(tokenizer.decode(hall_text))
        
    
#     print("########################################################\n\n")

In [21]:
# 1/0

In [22]:
# from transformers import (
#     AutoTokenizer,
#     AutoModelForCausalLM,
#     TrainingArguments,
#     BitsAndBytesConfig,
# )
# import torch
# from src.trainer import SelfCorrectionTrainer

In [23]:
# trainer = SelfCorrectionTrainer(model = "None")

In [24]:
# sample = train_dataset[0]
# sample["input_ids"] = torch.tensor(sample["input_ids"]).reshape(1, -1)
# sample["hallucination_labels"] = torch.tensor(sample["hallucination_labels"]).reshape(1, -1)
# sample["labels"] = torch.tensor(sample["labels"]).reshape(1, -1)
# print(sample)

In [25]:
# output = sample.copy()
# # output["logits"] = torch.tensor(output["input_ids"]).reshape(1, -1, 1)
# output["logits"] = torch.zeros(1, output["input_ids"].shape[-1], len(tokenizer.get_vocab())) - 100
# for i in range(output["input_ids"].shape[-1]):
#     output["logits"][0, i, output["input_ids"][0, i]] = 100
# output["hallucination_logits"] = torch.tensor(output["hallucination_labels"]).reshape(1, -1, 1)

In [26]:
# output["logits"][0, :-1, ...] = output["logits"].clone()[0, 1:, ...]

In [27]:
# trainer.compute_loss(sample, output, len(tokenizer.get_vocab()))

In [28]:
# import torch

# logits = torch.zeros(1, 10, 100)
# hallucination_logits = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]).reshape(1, 10, 1)

# additional_logits = torch.zeros_like(logits)
# additional_logits[:, :, -3:] = hallucination_logits
# logits = logits + additional_logits
# print(logits)

In [29]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
# base_model = AutoModelForCausalLM.from_pretrained(
#     "meta-llama/Llama-3.2-1B-Instruct",
# )

In [30]:
# print(base_model.lm_head.weight.shape)
# print(base_model.lm_head.weight[-5:, :10])

In [31]:
# base_model.resize_token_embeddings(len(tokenizer.get_vocab())+3)

In [32]:
# print(base_model.lm_head.weight.shape)
# print(base_model.lm_head.weight[-8:, :10])

In [33]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import os
# import sys
# import torch

# # Add the project root directory to the Python path
# project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
# if project_root not in sys.path:
#     sys.path.insert(0, project_root)

In [34]:
# path = "../../self-corrective-llama_untrained"
# # path = "meta-llama/Llama-3.2-1B-Instruct"

# tokenizer = AutoTokenizer.from_pretrained(path)
# tokenizer.pad_token = tokenizer.eos_token
# model = AutoModelForCausalLM.from_pretrained(path, trust_remote_code=True)

In [35]:
model

SelfCorrectiveLlama(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_em

In [36]:
import torch

def generate_with_hallucination_analysis(model, tokenizer, prompt_text, max_new_tokens=50):
    """
    Generates text and then runs a second pass to get hallucination logits
    for the generated tokens, avoiding interference with the generate loop.
    """
    # Ensure the model is in evaluation mode
    model.eval()
    
    # --- Pass 1: Generate the text ---
    # We use the standard, unmodified generate method.
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
    generated_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    
    # Decode the full generated text
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
    
    # --- Pass 2: Run a single forward pass on the full sequence to get all logits ---
    # This is more efficient than collecting them step-by-step.
    with torch.no_grad():
        full_sequence_outputs = model(generated_ids)
    
    # Extract the hallucination logits from the output
    hallucination_logits = full_sequence_outputs.hallucination_logits
    
    # We only care about the logits for the *newly generated* tokens.
    # The logit at position `i` is the prediction for the token at `i+1`.
    prompt_len = inputs.input_ids.shape[1]
    generated_hallucination_logits = hallucination_logits[:, prompt_len-1:-1, :]
    
    return generated_text, generated_hallucination_logits

# --- Example Usage ---
text, hall_logits = generate_with_hallucination_analysis(
    model, 
    tokenizer, 
    "What is the capital of France?",
    max_new_tokens=50
)
print("Generated Text:", text)
print("Hallucination Logits Shape:", hall_logits.shape)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


special_token_mask:
tensor([[False, False, False, False, False, False, False, False]])
main_logits:
tensor([[[ 2.8333,  3.5809,  7.0268,  ..., -1.2470, -1.2469, -1.2468],
         [ 6.5817,  4.0679,  2.5371,  ..., -1.2149, -1.2154, -1.2152],
         [ 2.7450,  3.9400,  2.1865,  ..., -2.5903, -2.5903, -2.5904],
         ...,
         [ 2.1232,  2.1697,  1.5211,  ..., -1.8789, -1.8788, -1.8786],
         [10.2963,  6.1449,  6.6862,  ...,  0.2313,  0.2310,  0.2312],
         [ 5.0945,  2.6669,  4.5030,  ..., -0.0846, -0.0852, -0.0852]]])
new_logits:
tensor([[[-0.0007, -0.0007, -0.0007],
         [-0.0037, -0.0037, -0.0037],
         [-0.0013, -0.0013, -0.0013],
         [-0.0045, -0.0045, -0.0045],
         [ 0.0033,  0.0033,  0.0033],
         [-0.0092, -0.0092, -0.0092],
         [ 0.0034,  0.0034,  0.0034],
         [ 0.0042,  0.0042,  0.0042]]])
logits:
tensor([[[ 2.8333e+00,  3.5809e+00,  7.0268e+00,  ..., -6.5300e-04,
          -6.5300e-04, -6.5300e-04],
         [ 6.5817e+00,  4.0

In [37]:
hall_logits

tensor([[[-1.6630e-01, -2.6610e-01,  5.5706e-01,  1.1075e-01],
         [ 1.7694e-01, -7.5278e-01,  1.2451e-01,  1.8640e-01],
         [-2.8399e-01, -9.9048e-01,  3.5317e-01, -4.3676e-01],
         [ 1.4748e-01, -2.3290e-01,  1.7004e-01, -4.4632e-01],
         [-1.7201e-03, -2.7841e-01,  9.5527e-01,  2.6030e-01],
         [-2.8993e-01,  2.2951e-01,  4.2268e-01, -1.5192e-01],
         [ 6.7953e-02, -4.7396e-01,  1.0069e+00, -3.4818e-02],
         [-5.8172e-01, -1.0271e-01,  3.2743e-01, -2.8937e-01],
         [-1.4972e-01, -6.8268e-01,  1.9739e-01,  2.0465e-01],
         [-2.5709e-01,  2.0654e-02,  1.9311e-01, -1.4697e-01],
         [-1.3482e-01, -7.7888e-01,  6.7434e-01, -1.9322e-01],
         [-1.2464e-01,  2.6087e-01,  1.3779e-01,  1.5636e-01],
         [-2.4819e-01,  3.7286e-01,  4.9692e-01, -1.9903e-01],
         [-3.5792e-01, -2.8828e-01,  3.9076e-01,  1.6015e-01],
         [ 2.2843e-01,  2.7824e-01, -7.4774e-02,  9.3958e-02],
         [ 1.4222e-01, -4.8559e-01,  2.3051e-01,  3.257

In [52]:
# p1 = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a specialized question-answering AI. Your task is to give a concise answer to the question using *only* the provided context. Make sure to always give an answer. Use <DEL_W>, <DEL_S> or <DEL_A> tokens if needed.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContext:\n'''\nSichuan has been historically known as the \"Province of Abundance\". It is one of the major agricultural production bases of China. Grain, including rice and wheat, is the major product with output that ranked first in China in 1999. Commercial crops include citrus fruits, sugar cane, sweet potatoes, peaches and grapes. Sichuan also had the largest output of pork among all the provinces and the second largest output of silkworm cocoons in 1999. Sichuan is rich in mineral resources. It has more than 132 kinds of proven underground mineral resources including vanadium, titanium, and lithium being the largest in China. The Panxi region alone possesses 13.3% of the reserves of iron, 93% of titanium, 69% of vanadium, and 83% of the cobalt of the whole country. Sichuan also possesses China's largest proven natural gas reserves, the majority of which is transported to more developed eastern regions.\n'''\n\nQuestion: What are the major agricultural outputs of Sichuan?<|eot_id|><|start_header_id|>assistant<|end_header_id|>",
p1 = "What is the capital of France?"
inputs = tokenizer(p1, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[128000,   3923,    374,    279,   6864,    315,   9822,     30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


In [53]:
# res = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], output_hallucination_logits=True)
res = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


special_token_mask:
tensor([[False, False, False, False, False, False, False, False]])
main_logits:
tensor([[[ 2.8333,  3.5809,  7.0268,  ..., -1.2470, -1.2469, -1.2468],
         [ 6.5817,  4.0679,  2.5371,  ..., -1.2149, -1.2154, -1.2152],
         [ 2.7450,  3.9400,  2.1865,  ..., -2.5903, -2.5903, -2.5904],
         ...,
         [ 2.1232,  2.1697,  1.5211,  ..., -1.8789, -1.8788, -1.8786],
         [10.2963,  6.1449,  6.6862,  ...,  0.2313,  0.2310,  0.2312],
         [ 5.0945,  2.6669,  4.5030,  ..., -0.0846, -0.0852, -0.0852]]])
new_logits:
tensor([[[-0.0007, -0.0007, -0.0007],
         [-0.0037, -0.0037, -0.0037],
         [-0.0013, -0.0013, -0.0013],
         [-0.0045, -0.0045, -0.0045],
         [ 0.0033,  0.0033,  0.0033],
         [-0.0092, -0.0092, -0.0092],
         [ 0.0034,  0.0034,  0.0034],
         [ 0.0042,  0.0042,  0.0042]]])
logits:
tensor([[[ 2.8333e+00,  3.5809e+00,  7.0268e+00,  ..., -6.5300e-04,
          -6.5300e-04, -6.5300e-04],
         [ 6.5817e+00,  4.0

In [54]:
model.original_vocab_size

128256

In [55]:
res

tensor([[128000,   3923,    374,    279,   6864,    315,   9822,     30,  12366,
            627,    791,   6864,    315,   9822,    374,  12366,     13,  12366,
            374,    279,   1455,  95551,   3363,    304,   9822,    323,    374,
           3967]])

In [56]:
tokens = res

In [57]:
# tokens, hall = res

In [58]:
tokenizer.decode(tokens[0])

'<|begin_of_text|>What is the capital of France? Paris.\nThe capital of France is Paris. Paris is the most populous city in France and is known'

In [44]:
1/0

ZeroDivisionError: division by zero

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

base_model_name = "meta-llama/Llama-3.2-1B-Instruct"
special_tokens = ["<DEL_W>", "<DEL_S>", "<DEL_A>"]
intermediate_save_path = "./temp_resized_model"

# --- Step 1: Load, Resize, and Save the STANDARD Llama model ---
print("--- Step 1: Loading and resizing original LlamaForCausalLM ---")
# Load the original, standard model class
original_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
original_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
)

# Add tokens and resize the embeddings of the standard model
original_tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})
original_model.resize_token_embeddings(len(original_tokenizer))

# Save this intermediate model. It is still a LlamaForCausalLM, just with a bigger lm_head
print(f"--- Saving resized LlamaForCausalLM to {intermediate_save_path} ---")
original_model.save_pretrained(intermediate_save_path)
original_tokenizer.save_pretrained(intermediate_save_path)

# Clear memory
del original_model
del original_tokenizer
torch.cuda.empty_cache()

--- Step 1: Loading and resizing original LlamaForCausalLM ---


KeyboardInterrupt: 

In [ ]:
# --- Step 2: Load the RESIZED model and run the prompt ---
print("\n--- Step 2: Loading the RESIZED LlamaForCausalLM and testing ---")
# Load the model you just saved. This is still the standard class.
resized_tokenizer = AutoTokenizer.from_pretrained(intermediate_save_path)
resized_model = AutoModelForCausalLM.from_pretrained(
    intermediate_save_path,
)

print(resized_model.lm_head.weight.shape)

prompt = "What is the capital of France?"
inputs = resized_tokenizer(prompt, return_tensors="pt")
outputs = resized_model.generate(**inputs, max_new_tokens=50)
print("--- Output from RESIZED LlamaForCausalLM ---")
print(resized_tokenizer.decode(outputs[0], skip_special_tokens=False))

# Clear memory
del resized_model
del resized_tokenizer
torch.cuda.empty_cache()


--- Step 2: Loading the RESIZED LlamaForCausalLM and testing ---


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


torch.Size([128259, 2048])
--- Output from RESIZED LlamaForCausalLM ---
<|begin_of_text|>What is the capital of France? Paris
The capital of France is Paris, which is also the most populous city in the country. Paris is known for its rich history, art, fashion, and cuisine. It is home to many famous landmarks, such as the Eiffel Tower


In [ ]:
# # --- Step 3 (Optional): Confirm your custom model behaves the same ---
# print("\n--- Step 3: Loading the RESIZED model into your custom class ---")
# # This is the final check. We load the same resized model, but now we
# # let it be interpreted as your custom class.
# # We must include the local modeling.py file via `trust_remote_code=True`.
# custom_tokenizer = AutoTokenizer.from_pretrained(intermediate_save_path)
# custom_model = AutoModelForCausalLM.from_pretrained(
#     intermediate_save_path,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
#     trust_remote_code=True # This will use your local `modeling.py`
# )

# inputs = custom_tokenizer(prompt, return_tensors="pt").to("cuda")
# outputs = custom_model.generate(**inputs, max_new_tokens=50)
# print("--- Output from RESIZED SelfCorrectiveLlama ---")
# print(custom_tokenizer.decode(outputs[0], skip_special_tokens=False))